# Real or Not? NLP with Disaster Tweets

Kaggle competition. Link: https://www.kaggle.com/c/nlp-getting-started/overview

## Table of Contents

1. [Set-up](#Set-up)
2. [Data Analysis](#Data-Analysis)
2. [Feature Engineering Functions](#Feature-Engineering-Functions)
4. [Data Preparation](#Data-Preparation)
5. [Scikit Learn Machine Learning](#Scikit-Learn-Machine-Learning)
6. [Keras Machine Learning](#Keras-Machine-Learning)

## Set-up

In [1]:
# Basics
import numpy as np
import pandas as pd
import re

In [2]:
# Feature Engineering
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
# Scikit Learn
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [4]:
# Keras
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

Using TensorFlow backend.


In [5]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', -1)

In [6]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

In [7]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


## Data Analysis

In [8]:
df_test = train_df.copy()

In [9]:
df_test.isnull().sum()

id          0   
keyword     61  
location    2533
text        0   
target      0   
dtype: int64

In [10]:
print(df_test['target'].value_counts())
print(df_test['target'].value_counts(1))

0    4342
1    3271
Name: target, dtype: int64
0    0.57034
1    0.42966
Name: target, dtype: float64


Let's look at hastags

In [11]:
df_test['hashtag'] = df_test['text'].apply(lambda tweet: '#' in tweet)
df_test.head()

,id,keyword,location,text,target,hashtag
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,True
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,False
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,False
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,True
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,True


In [12]:
print(df_test[df_test['hashtag']]['target'].value_counts(1))
print(df_test[~df_test['hashtag']]['target'].value_counts(1))

0    0.503123
1    0.496877
Name: target, dtype: float64
0    0.590567
1    0.409433
Name: target, dtype: float64


Let's look at if there's any punctuation (right now just . , ' " ; :)

In [13]:
df_test['punctuation'] = df_test['text'].apply(lambda tweet: '.' in tweet or ',' in tweet or "," in tweet or '"' in tweet or ';' in tweet or ':' in tweet)
df_test.head()

,id,keyword,location,text,target,hashtag,punctuation
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,True,False
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,False,True
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,False,True
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,True,True
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,True,False


In [14]:
df_test['text'].iloc[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [15]:
print(df_test[df_test['punctuation']]['target'].value_counts(1))
print(df_test[~df_test['punctuation']]['target'].value_counts(1))

0    0.519102
1    0.480898
Name: target, dtype: float64
0    0.756231
1    0.243769
Name: target, dtype: float64


Let's look at some numerical stuff

In [16]:
df_test['tweet length'] = df_test['text'].apply(lambda tweet: len(tweet))
df_test.head()

,id,keyword,location,text,target,hashtag,punctuation,tweet length
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,True,False,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,False,True,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,False,True,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,True,True,65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,True,False,88


In [17]:
df_test['number of words'] = df_test['text'].apply(lambda tweet: len(tweet.split(' ')))
df_test.head()

,id,keyword,location,text,target,hashtag,punctuation,tweet length,number of words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,True,False,69,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,False,True,38,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,False,True,133,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,True,True,65,9
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,True,False,88,17


In [18]:
df_test['number of sentences'] = df_test['text'].apply(lambda tweet: len(tweet.split('.')))
df_test.head()

,id,keyword,location,text,target,hashtag,punctuation,tweet length,number of words,number of sentences
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,True,False,69,13,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,False,True,38,7,2
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,False,True,133,22,2
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,True,True,65,9,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,True,False,88,17,1


In [19]:
df_test['text'].iloc[1]

'Forest fire near La Ronge Sask. Canada'

In [20]:
df_test.corr()['target']

id                     0.060781
target                 1.000000
hashtag                0.074486
punctuation            0.197150
tweet length           0.181817
number of words        0.040862
number of sentences    0.156265
Name: target, dtype: float64

In [21]:
df_test.corr()

,id,target,hashtag,punctuation,tweet length,number of words,number of sentences
id,1.000000,0.060781,-0.008026,0.012285,0.017393,0.005267,0.018247
target,0.060781,1.000000,0.074486,0.197150,0.181817,0.040862,0.156265
hashtag,-0.008026,0.074486,1.000000,0.126030,0.208775,0.055172,0.070796
punctuation,0.012285,0.197150,0.126030,1.000000,0.403420,0.168680,0.517039
tweet length,0.017393,0.181817,0.208775,0.403420,1.000000,0.831328,0.401347
number of words,0.005267,0.040862,0.055172,0.168680,0.831328,1.000000,0.223232
number of sentences,0.018247,0.156265,0.070796,0.517039,0.401347,0.223232,1.000000


In [22]:
# df_test[df_test['target']==0]['text'].sample(100)

In [23]:
# df_test[df_test['target']==1]['text'].sample(100)

## Feature Engineering Functions

In [24]:
def add_substring_in_tweet_column(substring, df):
    """Function will add new column to df.
    New column will be of boolean type: True if substring is in that row's tweet, False otherwise.
    Note: if substring is all lowercase, will assume case does not matter and so will search if substring is in
    lowercase version of tweet. If case matters, then substring must contain at least one capital. This is an assumption."""
    
    new_column_name = "'" + substring + "' in tweet?" # can tell if case matters if substring has capitals
    match_case = not substring == substring.lower()
    
    if match_case:
        df[new_column_name] = df['text'].apply(lambda tweet: substring in tweet)
    
    else:
        df[new_column_name] = df['text'].apply(lambda tweet: substring in tweet.lower())
        
# add_substring_in_tweet_column('.', df_test)
# add_substring_in_tweet_column('our', df_test)
# add_substring_in_tweet_column('fire', df_test)
# display(df_test.head())

In [25]:
def add_sentiment_analysis_column(df, method):
    """Takes in a dataframe, df, and adds sentiment analysis columns to it using a specified method.
    If no correct method is specified, nothing will be added to the dataframe.
    Columns add are: 'neg', 'neu', 'pos' and 'compound'.
    Possible methods: nltk_vader
    """
    
    if method == 'nltk_vader':
        sid = SentimentIntensityAnalyzer()
        df['sentiment'] = df['text'].apply(sid.polarity_scores)
        df['neg'] = df['sentiment'].apply(lambda sent: sent['neg'])
        df['neu'] = df['sentiment'].apply(lambda sent: sent['neu'])
        df['pos'] = df['sentiment'].apply(lambda sent: sent['pos'])
        df['compound'] = df['sentiment'].apply(lambda sent: sent['compound'])
        df.drop('sentiment', axis=1, inplace=True)
        
    else:
        print('No valid method entered - no new columns will be added to the dataframe')

In [26]:
def add_spacy_and_other_numeric_columns(df):
    """Adds columns to a dataframe, df, that use the library spacy.
    Since spacy provides some interesting numeric results which will be added as columns,
    this function will also add the other numeric columns as well, since they may depend on these results."""
    
    # First, create a doc column for each tweet (i.e. tokenize each tweet)
    #df['doc'] = df['text'].apply(lambda tweet: nlp(tweet))
    generator = nlp.pipe(df['text'])
    spacy_dict = {'number of tokens':[], 'number of sentences':[], 'number of words':[],
                  'number of nouns':[], 'number of verbs':[], 'number of adjectives':[],
                  'number of adverbs':[], 'number of pronouns':[], 'number of punctuation':[]}
    
    for doc in generator:
        # Number of tokens (not sure if it's that useful)
        spacy_dict['number of tokens'].append(len(doc))
        
        # Number of sentences
        num_sentences = len([sentence for sentence in doc.sents])
        spacy_dict['number of sentences'].append(num_sentences)
        
        # Number of words (including stop words)
        num_words = len([token.text for token in doc if token.is_punct != True])
        spacy_dict['number of words'].append(num_words)

        # Number of nouns (use first line if don't want stop words or punctuations)
        #num_nouns = len([token.text for token in doc if token.is_stop != True and token.is_punct != True and token.pos_ == "NOUN"])
        num_nouns = len([token.text for token in doc if token.pos_ == "NOUN"])
        spacy_dict['number of nouns'].append(num_nouns)
        
        # Number of verbs
        num_verbs = len([token.text for token in doc if token.pos_ == "VERB"])
        spacy_dict['number of verbs'].append(num_verbs)
        
        # Number of adjectives
        num_adj = len([token.text for token in doc if token.pos_ == "ADJ"])
        spacy_dict['number of adjectives'].append(num_adj)
        
        # Number of adverbs
        num_adverbs = len([token.text for token in doc if token.pos_ == "ADV"])
        spacy_dict['number of adverbs'].append(num_adverbs)
        
        # Number of pronouns
        num_pronouns = len([token.text for token in doc if token.pos_ == "PRON"])
        spacy_dict['number of pronouns'].append(num_pronouns)
        
        # Number of punctuation symbols used 
        # (Note: not sure if hastags are included - first tweet hashtag is considered a noun)
        # Could maybe use is_punct instead??
        num_punct = len([token.text for token in doc if token.pos_ == "PUNCT"])
        spacy_dict['number of punctuation'].append(num_punct)
        
    
    spacy_df = pd.DataFrame(data=spacy_dict)
    df = pd.concat([df, spacy_df], axis=1)
    
    
    df['tweet length'] = df['text'].apply(lambda tweet: len(tweet))
    df['word length (including whitespace)'] = df['tweet length'] / df['number of words']
    df['word per sentence'] = df['number of words'] / df['number of sentences']
    df['total hastags'] = df['text'].apply(lambda tweet: len(re.findall('#', tweet)))
    
    return df

In [27]:
def add_vectorization_columns(df, method='tfidf', min_df=0.0, max_df=1.0, stop_words='english'):
    """Adds vectorization columns to dataframe(s) based on tweets.
    
    Notes:
    -Possible methods are 'count' (CountVectorizer) and 'tfidf' (TfidfVectorizer)
    -Can also specify min_df, max_df and stop_words
    """
    
    try:     
        # Set up vectorization
        if method == 'count':
            vectorizer = CountVectorizer(min_df=min_df, max_df=max_df,stop_words=stop_words)

        if method == 'tfidf':
            vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df,stop_words=stop_words)        

        # Add vectors to df and return
        vectors = vectorizer.fit_transform(df["text"])
        vectors = pd.DataFrame(vectors.todense())
        df = pd.concat([df, vectors], axis=1)
        return df
        
    except:
        print('No valid method entered!')

        
# add_vectorization_columns(train_df,'tfidf',50,0.9,'english')

## Data Preparation

In [66]:
cols_to_drop = ['id', 'keyword', 'location', 'text']
substring_list = ['#', '.', ',', ':', "'", 'http']
spacy_cols = True
spacy_and_num_cols = True
sentiment_analysis_method = 'nltk_vader'
encode_keyword = False # Haven't found this to be too useful

vectorize_method = 'tfidf' # very slow
min_df = 2 #0.0
max_df = 1.0
stop_words = 'english'

ml_model = 'ridge'
nearest_neighbors = 1

nn_hidden_layers = 2
nn_num_neurons = 2000

In [67]:
def prepare_dataframe_for_machine_learning(dataframe=train_df, vectorization=False):
    """Prepares a dataframe for machine learning.
    Based on constants specified in the cell above, this function may add columns to the dataframe based on:
    keyword search, numeric calculations, sentiment analysis, one hot encoding the keywords,
    and/or vectorizing the words.
    It wil then drop columns not needed for machine learning.
    
    Note: Need to pass in a boolean vectorization variable because we can't set up X_test by just calling this
    function as vectors will differ with those of X_train (as different tweets have different words).
    Vectorization will be kept as part of this function for EDA purposes. However, when preparing submission,
    vectorization must be performed separately (and we'd need to pass in vectorization=False)
    """
    
    # Prepare a copy, so that experimenting is easy
    df = dataframe.copy()
    
    # Add substring columns
    for substring in substring_list:
        add_substring_in_tweet_column(substring, df)
        
    # Add spacy and numeric columns, if true
    # Note: have to write df = f(df) for this one as it won't alter the columns like the other functions
    if spacy_cols:
        df = add_spacy_and_other_numeric_columns(df)
    
    # Add sentiment analysis
    add_sentiment_analysis_column(df, method=sentiment_analysis_method)
    
    # Encode keyword, if true
    if encode_keyword:
        encoded_keys = pd.get_dummies(df['keyword'])
        df = pd.concat([df, encoded_keys], axis=1)
        
    # Vectorize tweets, if true
    if vectorization:
        df = add_vectorization_columns(df=df, method=vectorize_method, min_df=min_df, max_df=max_df, stop_words=stop_words)
    
    
    # Drop columns not wanted for machine learning
    df.drop(cols_to_drop, axis=1, inplace=True)
    
    # Return prepared dataframe
    return df

prepared_df = prepare_dataframe_for_machine_learning(train_df, vectorization=True)
display(prepared_df.head())
#display(prepared_df.corr()['target'])

,target,'#' in tweet?,'.' in tweet?,"',' in tweet?",':' in tweet?,''' in tweet?,'http' in tweet?,number of tokens,number of sentences,number of words,number of nouns,number of verbs,number of adjectives,number of adverbs,number of pronouns,number of punctuation,tweet length,word length (including whitespace),word per sentence,total hastags,neg,neu,pos,compound,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,...,6281,6282,6283,6284,6285,6286,6287,6288,6289,6290,6291,6292,6293,6294,6295,6296,6297,6298,6299,6300,6301,6302,6303,6304,6305,6306,6307,6308,6309,6310,6311,6312,6313,6314,6315,6316,6317,6318,6319,6320,6321,6322,6323,6324,6325,6326,6327,6328,6329,6330
0,1,True,False,False,False,False,False,14,2,13,4,3,0,0,1,0,69,5.307692,6.5,1,0.000,0.851,0.149,0.2732,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,False,True,False,False,False,False,8,2,7,2,0,0,0,0,1,38,5.428571,3.5,0,0.286,0.714,0.000,-0.3400,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,False,True,False,False,True,False,25,2,22,7,4,1,0,0,3,133,6.045455,11.0,0,0.095,0.905,0.000,-0.2960,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,True,False,True,False,False,False,9,1,8,5,1,0,0,0,0,65,8.125000,8.0,1,0.000,1.000,0.000,0.0000,0.0,0.420625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,True,False,False,False,False,False,18,1,16,4,3,0,1,0,0,88,5.500000,16.0,2,0.000,1.000,0.000,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Scikit Learn Machine Learning

In [64]:
def evaluate_model_and_prepare_file_skl(prepared_dataframe=prepared_df, ml_model=ml_model, submission=False, 
                                        train_df=train_df, test_df=test_df):
    """Takes in a prepared dataframe and a scikit learn machine learning model, and returns the F1 and accuracy 
    cross-validation scores for when that model is fit to the prepared dataframe. 
    (Uses a prepared dataframe to save time)
    
    If submission is set to true, it will fit the model and prepare the file to submit in the format 
    required by the contest (scores will not be returned). This will use the original, not prepared
    dataframes train_df and test_df.
    
    If no correct model is specified, no scores will be returned.
    
    Possible ml_models: ridge (Ridge Classifier), knn (K Nearest Neighbors), dt (Decision Tree), xgb (XGBoost)
    nb (Naive Bayes - all values must be non-negative), lr (Logistic Regression), svm (Support-Vector Machine),
    nn (Neural Network - scikit learn's version)
    """
    
    # Set up machine learning model based on ml_model
    
    if ml_model == 'ridge':
        clf = RidgeClassifier()
    
    if ml_model == 'knn':
        clf = KNeighborsClassifier(n_neighbors=nearest_neighbors)
        
    if ml_model == 'dt':
        clf = DecisionTreeClassifier()
        
    if ml_model == 'nb':
        clf = MultinomialNB()
    
    if ml_model == 'xgb':
#         params = {'objective': 'binary:logistic', 'max_depth': 2, 'learning_rate': 1.0, 
#                   'silent': True, 'n_estimators': 5}
#         clf = XGBClassifier(**params)
        clf = XGBClassifier()
    
    if ml_model == 'lr':
        clf = LogisticRegression(solver='liblinear',max_iter=1000)
        
    if ml_model == 'svm':
        clf = SVC(gamma='auto')
    
    if ml_model == 'nn':
        clf = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                            hidden_layer_sizes=(nn_num_neurons, nn_hidden_layers), 
                            random_state=1)
    
    
    try:    
        
        if submission:
            # Prepare file for submission

            # Make a copy of the dataframes
            train = train_df.copy()
            test = test_df.copy()


            # Vectorize, if desired, first so that we can properly fit test
            if vectorize_method:
                # Set up vectorizer
                if vectorize_method == 'count':
                    vectorizer = CountVectorizer(min_df=min_df, max_df=max_df, stop_words=stop_words)
                if vectorize_method == 'tfidf':
                    vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, stop_words=stop_words)  

                # Create vectors and add to each dataframe
                train_vectors = pd.DataFrame(vectorizer.fit_transform(train["text"]).todense())
                test_vectors = pd.DataFrame(vectorizer.transform(test["text"]).todense())
                train = pd.concat([train, train_vectors], axis=1)
                test = pd.concat([test, test_vectors], axis=1)


            # Prepare the dataframes for machine learning
            prepared_train = prepare_dataframe_for_machine_learning(train, vectorization=False)
            prepared_test = prepare_dataframe_for_machine_learning(test, vectorization=False)


            # Perform machine learning and save the submission file
            clf.fit(prepared_train.drop("target", axis=1), prepared_train["target"])
            sample_submission = pd.read_csv("../data/sample_submission.csv")
            sample_submission["target"] = clf.predict(prepared_test)
            sample_submission.to_csv("../submission.csv", index=False)


        else: 
            # Return f1 scores
            df = prepared_dataframe.copy()
            f1_scores = cross_val_score(clf, df.drop("target", axis=1),
                                                        df["target"], cv=3, scoring="f1")
            accuracy_scores = cross_val_score(clf, df.drop("target", axis=1),
                                                              df["target"], cv=3, scoring="accuracy")
            print('F1 Scores:')
            print(f1_scores)
            print()
            print('Accuracy Scores:')
            print(accuracy_scores)
    
    
    except:
        print('No valid model entered/incorrect vectorization method!')
        

model_list = ['ridge', 'knn', 'dt', 'xgb', 'lr', 'svm', 'nn']
model_list = ['ridge']
for model in model_list:
    print('Model:', model)
    evaluate_model_and_prepare_file_skl(prepared_dataframe=prepared_df, ml_model=model, submission=False)
    print()

Model: ridge
F1 Scores:
[0.64648538 0.5967118  0.68393782]

Accuracy Scores:
[0.72863332 0.67126527 0.73551439]



In [31]:
# evaluate_model_and_prepare_file_skl(train_df=train_df, test_df=test_df, ml_model=ml_model, submission=True)

Note: Model performs best with vectorization. ML model that performs best with vectorization is the ridge classifier and logistic regression. I haven't formally tested it, but they seem efficient too, so I'll just look at those for now until I add neural nets.

Update: Experimented with neural nets. Scikit learn's version does not perform as well as some of the other models. However, Keras' does. Since the above function is specific to scikit learn models, I'll create a new function for Keras.

## Keras Machine Learning

In [ ]:
def evaluate_model_and_prepare_file_keras(train_df=train_df, test_df=test_df, 
                                          epochs=10, neurons=nn_num_neurons, hidden_layers=nn_hidden_layers):
    """Similar to evaluate_model_and_prepare_file_skl except this one will only train a Keras neural network.
    Unlike the previous one, this function will take in the dimensions of the neural network and the number of
    epochs as this function can be more specific.
    
    As training a neural network is very slow, unlike the previous function, this one will both print out
    the confusion matrix and classification report, and prepare the file for submission. To do this,
    it will prepare the dataframes for machine learning and won't take in a prepared dataframe.
    """

    # First, prepare the dataframes for machine learning
    
    # Make a copy of the dataframes
    train = train_df.copy()
    test = test_df.copy()
    
    # Vectorize, if desired, first so that we can properly fit test
    if vectorize_method:
        # Set up vectorizer
        if vectorize_method == 'count':
            vectorizer = CountVectorizer(min_df=min_df, max_df=max_df, stop_words=stop_words)
        if vectorize_method == 'tfidf':
            vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, stop_words=stop_words)  

        # Create vectors and add to each dataframe
        train_vectors = pd.DataFrame(vectorizer.fit_transform(train["text"]).todense())
        test_vectors = pd.DataFrame(vectorizer.transform(test["text"]).todense())
        train = pd.concat([train, train_vectors], axis=1)
        test = pd.concat([test, test_vectors], axis=1)

    # Use the function to finish preparing the dataframes
    prepared_train = prepare_dataframe_for_machine_learning(train, vectorization=False)
    prepared_test = prepare_dataframe_for_machine_learning(test, vectorization=False)

    
    # Next, set up the neural network
    
    # Separate X and y, and make y categorical
    X = prepared_train.drop("target", axis=1)
    y = to_categorical(prepared_train["target"])

    # We also need to scale the data to help the performance of the model
    scaler_object = MinMaxScaler()
    scaler_object.fit(X)
    X = scaler_object.transform(X)
    X_submission = scaler_object.transform(prepared_test)
    
    # Set up the neural network. These features can be adjusted later.    
    model = Sequential()
    for i in range(0, hidden_layers):
        model.add(Dense(neurons, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    
    # Perform machine learning and save the submission file
    model.fit(X, y, epochs=epochs, verbose=2)
    sample_submission = pd.read_csv("../data/sample_submission.csv")
    sample_submission["target"] = model.predict_classes(X_submission)
    sample_submission.to_csv("../submission.csv", index=False)
    
    # May add these later, but obviously need to do train test split
    # Right now using my submissions for testing so I don't need it
    # print(confusion_matrix(y_test.argmax(axis=1),predictions))
    # print(classification_report(y_test.argmax(axis=1),predictions))


evaluate_model_and_prepare_file_keras(train_df=train_df, test_df=test_df, 
                                      epochs=10, neurons=2*prepared_df.shape[1], hidden_layers=2)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 12710)             80772050  
_________________________________________________________________
dense_26 (Dense)             (None, 12710)             161556810 
_________________________________________________________________
dense_27 (Dense)             (None, 2)                 25422     
Total params: 242,354,282
Trainable params: 242,354,282
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


In [38]:
# df = prepared_df.copy()

# X = df.drop("target", axis=1)
# y = to_categorical(df["target"])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# scaler_object = MinMaxScaler()
# scaler_object.fit(X_train)
# scaled_X_train = scaler_object.transform(X_train)
# scaled_X_test = scaler_object.transform(X_test)

# dim = X.shape[1]

# model = Sequential()
# model.add(Dense(2*dim, input_dim=dim, activation='relu'))
# model.add(Dense(2*dim, input_dim=dim, activation='relu'))
# model.add(Dense(2, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.summary()

# model.fit(scaled_X_train,y_train,epochs=10, verbose=2)

# predictions = model.predict_classes(scaled_X_test)

# # print(confusion_matrix(y_test.argmax(axis=1),predictions))
# # print(classification_report(y_test.argmax(axis=1),predictions))